In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [2]:
data = pd.read_csv('adult_cleaned_final.csv')

In [5]:
data = pd.get_dummies(data, drop_first=True)

In [6]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(data)

In [7]:
def evaluate_kmeans(features):
    selected_features = np.where(features > 0.5, 1, 0)
    X_selected = X_scaled[:, selected_features == 1]
    
    if X_selected.shape[1] == 0:
        return 1.0
    
    kmeans = KMeans(n_clusters=2, random_state=42)
    kmeans.fit(X_selected)
    labels = kmeans.labels_
    
    score = silhouette_score(X_selected, labels)
    return -score

In [8]:
def firefly_algorithm(dim, lb, ub, n=50, max_gen=100, alpha=0.5, beta=0.2, gamma=1.0):
    def attractiveness(distance):
        return beta * np.exp(-gamma * (distance ** 2))
 
    fireflies = np.random.uniform(lb, ub, (n, dim))
    fitness = np.array([evaluate_kmeans(f) for f in fireflies])
    
    for gen in range(max_gen):
        for i in range(n):
            for j in range(n):
                if fitness[i] > fitness[j]:
                    r = np.linalg.norm(fireflies[i] - fireflies[j])
                    fireflies[i] += attractiveness(r) * (fireflies[j] - fireflies[i])
                    fireflies[i] += alpha * (np.random.rand(dim) - 0.5)
                    fireflies[i] = np.clip(fireflies[i], lb, ub)
                    fitness[i] = evaluate_kmeans(fireflies[i])
                    
    best_idx = np.argmin(fitness)
    return fireflies[best_idx], fitness[best_idx]

In [9]:
dim = X_scaled.shape[1]
lb = [0] * dim
ub = [1] * dim

In [10]:
best_pos_firefly, best_fit_firefly = firefly_algorithm(dim, lb, ub)

c:\Users\DT User3\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\DT User3\AppData\Local\Programs\Python\Python310\lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] The system cannot find the file specified
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\Users\DT User3\AppData\Local\Programs\Python\Python310\lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
  File "c:\Users\DT User3\AppData\Local\Programs\Python\Python310\lib\subpro

In [ ]:
selected_features_firefly = np.where(best_pos_firefly > 0.5, 1, 0)

In [ ]:
print("Firefly selected features:", selected_features_firefly)

In [ ]:
print("Best silhouette score (Firefly):", -best_fit_firefly)